In [15]:
%pylab inline

import pandas as pd
import pickle as pk
from scipy import sparse as sp

Populating the interactive namespace from numpy and matplotlib


In [33]:
df =pd.read_csv(r'C:\Users\Ajay\Documents\Code\EXL\Projects\Retail Pharmacy\TextClustering\Data\2021VAERSDATA.csv',encoding='latin1')
docs = array(df['SYMPTOM_TEXT'])

In [17]:
docs[0]

'Right side of epiglottis swelled up and hinder swallowing pictures taken Benadryl Tylenol taken'

In [36]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
df=df[df['SYMPTOM_TEXT'].notna()]
docs = array(df['SYMPTOM_TEXT'])
def docs_preprocessor(docs):
    try:
        tokenizer = RegexpTokenizer(r'\w+')
        for idx in range(len(docs)):
            
            docs[idx] = docs[idx].lower()  # Convert to lowercase.
            docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

        # Remove numbers, but not words that contain numbers.
        docs = [[token for token in doc if not token.isdigit()] for doc in docs]

        # Remove words that are only one character.
        docs = [[token for token in doc if len(token) > 3] for doc in docs]

        # Lemmatize all words in documents.
        lemmatizer = WordNetLemmatizer()
        docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
    except:
        print(docs[idx-1],docs[idx])
  
    return docs

In [35]:
import pandas as pd
import numpy as np
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os

import nltk

from sklearn.metrics import confusion_matrix,classification_report
import nltk

from nltk import word_tokenize
from nltk.corpus import stopwords
import string

from nltk import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import re
# NLP text tokens
df=df[df['SYMPTOM_TEXT'].notna()]
def tokenize(text):
    words=word_tokenize(text)
    return words

# removing stopwords
stop=stopwords.words('english')
puncs=list(string.punctuation)
stop=stop+puncs
def remove_stops(words):
    cleaned=[w for w in words if not w.lower() in stop]
    return cleaned
# document parsing
def pos_tg(cleaned):
    pos=np.array(pos_tag(cleaned))
    return pos

def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
# lemitizing
lemmatizer=WordNetLemmatizer()

def lemmatize(pos):
    lemmatized=[]
    for w in pos:
        lemmatizedword=lemmatizer.lemmatize(w[0],get_simple_pos(w[1]))
        lemmatized.append(lemmatizedword)
    return lemmatized
def preprocess(text):
    words=tokenize(text)
    cleaned=remove_stops(words)
    pos=pos_tag(cleaned)
    lemmatized=lemmatize(pos)
    preprocessed=" ".join(lemmatized)
    return preprocessed

# data preprocessing step where some characters which are not required are removed 
# 1. special characters
# 2. digits in this use case
# 3. extra spaces
# 4. some words like block are used in lot of sentences and is surrounded by other characters ..needed to be separated
# 5 words less than  3 and greater than 20 add no meaning to data , so those are also removed
worddic={}
rows=[]
for i in df.index:
    row=df['SYMPTOM_TEXT'][i]
    try:
        row=df['SYMPTOM_TEXT'][i].lower()
    except:
        print(row)
    
    row=re.sub('[^A-Za-z]+', ' ', row)
    tokens=row.split(" ")
    
    tokens=[t.lower() for t in tokens if len(t)>=3 and len(t)<20]
    text=" ".join(tokens)

    row=preprocess(text)
    tokens=row.split(" ")
    for j in tokens:
        worddic[j]=worddic.get(j,0)+1
    rows.append(row)
    

nan


TypeError: expected string or bytes-like object

In [37]:
docs = docs_preprocessor(docs)
# docs=rows

In [38]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [39]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))


Number of unique words in initital documents: 27693
Number of unique words after removing rare and common words: 9095


In [40]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [37]:
dictionary

In [41]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 9095
Number of documents: 34119


In [42]:
from gensim.models import LdaModel

In [43]:
dictionary.id2token

{}

In [44]:
num_topics = 4
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 9min 40s


In [47]:
pip install pyLDAvis

  Using cached pyLDAvis-3.3.1.tar.gz (1.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 92.7/92.7 kB 2.6 MB/s eta 0:00:00
     ------------------------------------- 829.2/829.2 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached funcy-1.17-py2.py3-none-any.whl (33 kB)
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136904 sha256=e4afecdbf2b28cf6102ed89980939edf0f10126c5617118bfc4d724460261177
  Stored in directory: c:\users\ajay\appda

In [48]:
import pyLDAvis.gensim_models

In [52]:
p=pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

C:\Users\Ajay\Anaconda3\envs\python3-directml\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [53]:
pyLDAvis.save_html(p, 'lda.html')

In [50]:
import pyLDAvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
from tensorflow.python.client import device_lib
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:DML:0', device_type='DML')]

In [4]:
tf.test.is_gpu_available()

True

In [6]:
devices=device_lib.list_local_devices()

In [9]:
tf.compat.v1.config.experimental.set_visible_devices=devices[1]

In [10]:
tf.compat.v1.config.experimental.get_visible_devices

<function tensorflow.python.framework.config.get_visible_devices(device_type=None)>

In [11]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))



Device mapping:
/job:localhost/replica:0/task:0/device:DML:0 -> {"name": "AMD Radeon(TM) Graphics", "vendor_id": 4098, "device_id": 5708, "driver_version": "30.0.13032.0"}



In [13]:
import tensorflow as tf

In [14]:
tf.test.gpu_device_name()

'/device:DML:0'